<a href="https://colab.research.google.com/github/hariseldon99/msph402b/blob/main/quantum_HO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quantum Oscillator with NumPy

In this notebook we discuss a bit the basics of numerical methods and plotting in python by looking at a simple physics problem. Importantly, we will not reinvent the wheel, but use some standard libraries for scientific programming. They will be:

- The [jupyter notebook](https://jupyter.org/), for simple interactive coding.
- [Numpy](https://numpy.org/) for all kinds of numerical methods.
- [Matplotlib](https://matplotlib.org/) for all kinds of plotting.

## Harmonic Oscillator
Consider the Hamiltonian of the Quantum Harmonic Oscillator
\begin{equation*}
H = \frac{1}{2} \left(x^2 + p^2\right).
\end{equation*}
Here, $H$ is the Hamiltonian operator for the harmonic oscillator, and $x,p$ are the position and momentum operators, respectively. We have set the usual constants $\hbar=m=\omega=1$. Thus, the quantum canonicality of position and momentum becomes $\left[x,p\right]\equiv xp-px = i$. Now, if we define annihilation and creation operators,
\begin{align*}
a &= \frac{1}{\sqrt{2}}\left(x + ip\right),\\
a^\dagger &= \frac{1}{\sqrt{2}}\left(x - ip\right),
\end{align*}
then $H = a^\dagger a + \frac{1}{2}$. The eigenvalues of $H$ are given by $n+\frac{1}{2}$, with non-negative integer values $n=0,1,2,\dots \infty$. The corresponding eigenvectors $\left\vert n \right\rangle$ can be built by successive applications of the creation operator on the ground state $\left\vert 0 \right\rangle$, since
\begin{equation*}
a^\dagger \left\vert n \right\rangle = \sqrt{n+1}\;\left\vert n+1 \right\rangle.
\end{equation*}
For further details, consult any textbook on quantum mechanics, such as [Cohen Tannoudji *et. al.*, *Quantum Mechanics Vol. 1*](https://bit.ly/cohen-tannoiudjivol1), or [Sengupta and Pal, *Introduction to Quantum Mechanics*](https://bit.ly/krishnenduda).

Let us verify this result computationally. One problem is that the harmonic oscillator has infinite number of eigenstates, and a infinite dimensional space cannot be represented in finite-sized computer memory. Instead, let us just choose the first $4$ states, namely $\left\vert 0 \right\rangle, \left\vert 1 \right\rangle, \left\vert 2 \right\rangle, \left\vert 3 \right\rangle$. The matrix elements of the creation operator are given by $\left(a^\dagger\right)_{mn} = \left\langle m\right\vert a^\dagger \left\vert n \right\rangle =\sqrt{n+1}\;\delta_{m,n+1}$. Thus, the **matrix representation** of the annihilation operator would look like this:
\begin{equation*}
a^\dagger = \begin{pmatrix}
            0 & 0 & 0 & 0 \\
            \sqrt{1} & 0 & 0 & 0 \\
            0 & \sqrt{2} & 0 & 0 \\
            0 & 0 & \sqrt{3} & 0             
            \end{pmatrix}.
\end{equation*}

How can we make this matrix in `NumPy`? Let us first create an array `[1,2,3]`

In [ ]:
import numpy as np
np.array([1,2,3])

We can also create this array without creating a python list first. 

In [ ]:
N=4
np.arange(1,N)

Next, take square root of all elements by applying the function `numpy.sqrt()` to the whole array

In [ ]:
np.sqrt(np.arange(1,N))

This array now needs to be the lower subdiagonal of $a^\dagger$. For this, we can use the `numpy.diag()` function.

In [ ]:
a_dag = np.diag(np.sqrt(np.arange(1,N)), k=-1)
a_dag

Now, recall that 
\begin{align*}
H &= a^\dagger a + \frac{1}{2}\\
    & = \left(a^\dagger\right) \cdot \left(a^\dagger\right)^\dagger + 0.5 \times \hat{1}. 
\end{align*}
Here, the '$\cdot$' refers to matrix-matrix multiplication. The identity operator $\hat{1}$ has a matrix representation that is given by the identity matrix, which can be created using the `numpy.eye()` function. Thus,

In [ ]:
a = a_dag.T.conj()
H =  (a_dag @ a ) + 0.5 * np.eye(N)
H

Here, we have used the `NumPy` idiom `@` to denote matrix multiplication. As you can clearly see, $H$ is diagonal in this basis, and the first four eigenvalues are displayed. We can trivially verify this by using the `numpy.linalg.eigvals()` function, yielding

In [ ]:
from numpy import linalg
evals = linalg.eigvals(H)
evals

The ground state is given by the smallest energy. Remember that each `NumPy` array has a `.min()` method that yields the smallest value in that array. Thus,

In [ ]:
ground_state = evals.min()
ground_state

Let us do this with more levels. If we repeat all previous steps together and setting $N=40$ (say), we have

In [ ]:
import numpy as np
from numpy import linalg
N=40
a_dag = np.diag(np.sqrt(np.arange(1,N)), k=-1)
a = a_dag.conj().T
H = 0.5 * np.eye(N) + (a_dag @ a )
evals = linalg.eigvals(H)
evals

Now, suppose we want to plot these eigenvalues as a function of $n$. We will use a new python module, called `matplotlib`, in order to do this. First, let us import a special submodule from inside `matplotlib` called `pyplot` that has less functionality than the full `matplotlib`, but is easier to use.

In [ ]:
import matplotlib.pyplot as plt

This imports `pyplot` into a smaller name `plt`. Now, if we want to plot the eigenvalues $E_n$ versus $n$, we simply provide the x-axis data (a range of integers $n$) and the y-axis data (the `evals` array), as arguments to the `pyplot.plot()` function.

In [ ]:
x = np.arange(N)
y = evals
plt.plot(x,y) #The syntax is plt.plot(x_array, y_array)

## Anharmonic Oscillator

Of course, all this was super easy to derive analytically, so the numerical analysis is trivial. However, suppose we added a  **anharmonic** quartic potential to the Hamiltonian, yielding a new Hamiltonian $H^{(a)}$, where
\begin{equation*}
H^{(a)} = \frac{p^2}{2} + \frac{x^2}{2} + \alpha x^4 = H +  \alpha x^4 = a^\dagger a + \frac{1}{2} + \alpha x^4.
\end{equation*}
Now, no *exact* analytical solution to the eigenvalue problem is possible! Nonetheless, we can use perturbation theory to get approximate solutions, provided the perturbation $\alpha\ll 1$. Let us focus only on the ground state $E_0$ of $H^{(a)}$. A detailed calculation for $E_0$ was published by Carl Bender and Tai-Tsun Wu in the sixties (*Physical Review Letters* Vol. 185, No. 5, page 1231, 1969). Their result is reproduced below:
\begin{equation*}
E_0 = \frac{1}{2} + \frac{3}{4}\alpha -\frac{21}{8}\alpha^2 +\frac{333}{6}\alpha^3+ \dots.
\end{equation*}
You can see that, if the perturbation $\alpha=0$, the result yields the unperturbed energy of the ground state **i.e.** $1/2$.


Let us test this against numerical results. Recall that
\begin{align*}
a &= \frac{1}{\sqrt{2}}\left(x + ip\right),\\
a^\dagger &= \frac{1}{\sqrt{2}}\left(x - ip\right).
\end{align*}
Therefore, we can easily see that the position operator $x = \left(a + a^\dagger\right)/\sqrt{2}$. Thus,


In [ ]:
x_op =  (a + a_dag)/np.sqrt(2)
x_op

Recalling that '$\cdot$' means matrix multiplication, we can now write $\alpha x^4 = \alpha \times \left(x \cdot x \cdot x \cdot x\right)$, and use `@` idiom in `NumPy` to perform repeated matrix multiplications. Setting $\alpha$ to a small value, say, $0.01$, we have

In [ ]:
alpha = 0.01
H = 0.5 * np.eye(N) + (a_dag @ a ) + alpha * (x_op @ x_op @ x_op @ x_op)
ground_state = linalg.eigvals(H).min()
ground_state

Let us do this for a whole range of $\alpha$ s, starting from the beginning. First, let us use the `numpy.linspace()` function to create a uniformly spaced array of $100 \alpha$ values from $0-1$.

In [ ]:
alpha_vals = np.linspace(0.0, 0.1, 100)
alpha_vals

Nest, we use a `for` loop to populate an ampty list with the ground states for every value of $\alpha$ in the linespace. Starting from the very beginning, this yields,

In [ ]:
N=40
a_dag = np.diag(np.sqrt(np.arange(1,N)), k=-1) #The creation operator for the unperturbed system
a = a_dag.conj().T
x_op =  (a + a_dag)/np.sqrt(2)

gstates = [] #This empty list will be populated with the ground state energies for each value of alpha

for alpha in alpha_vals:
  H = 0.5 * np.eye(N) + a_dag @ a  + alpha * (x_op @ x_op @ x_op @ x_op)
  gstates.append(linalg.eigvals(H).min())

Let us now use `pyplot.plot()` to plot the ground state as a function of the chosen $\alpha$ values. This is easy enough, and can be done with just one line. Recall the syntax of `pyplot.plot()` as `plt.plot(x_array, y_array)`. This yields

In [ ]:
plt.plot(alpha_vals, gstates) 

LEt us now plot the analytical result. The analytical formula of Bender and Wu, to lowest perturbative order, was $\frac{1}{2} + \frac{3}{4}\alpha = 0.5 + 0.75\times\alpha$. Since we have a range of $\alpha$ s inside a `NumPy` array, we can simply use **array programming** rules, apply the analytical formula to the whole array, and plot the resultant array against the array of $\alpha$ s. This yields

In [ ]:
plt.plot(alpha_vals, 0.5  + 0.75 * alpha_vals)

How well do the analytical and numerical results match? Let us plot them together. This can be done by simply calling `pyplot.plot() ` twice, once for each $(x,y)$ dataset. We will first use the lowest-order perturbative result. In order to clearly distinguish between the analytical and numerical datasets, we will tell `pyplot.plot()` to use specific colors for each dataset with the optional keyword argument `color=`. This yields

In [ ]:
plt.plot(alpha_vals, gstates, color="blue") # The numerical data will be plotted in blue
plt.plot(alpha_vals, (1/2) + (3/4) * alpha_vals, color="red") #The lowest-order analytical data will be plotted in red

The "blue" color is the numerical result, and the "red" the perturbative result. As you can plainly see, the agreement is **asymptotic**, in that they approach each other as perturbation becomes smaller. This is an important aspect of perturbation theory. It is only accurate for small perturbations unless you calculate terms to all infinity orders.

We can confirm this by comparing the first, second, and third order with the numerical result. Since we have to compare many curves, it makes sense to label each curve, then have the labels in a plot *legend*. The `pyplot.plot` function has an additional optional keyword argument called `label=` that allows for labelling each plot. The legend can be created with the `pyplot.legend()` function.

In [ ]:
plt.plot(alpha_vals, gstates, color="blue", label="Numerical") #Numerical
plt.plot(alpha_vals, (1/2) + (3/4) * alpha_vals , color="red", label="First Order") #First order
plt.plot(alpha_vals, (1/2) + (3/4) * alpha_vals - (21/8) * alpha_vals**2, color="green", label="Second Order") #Second order
plt.plot(alpha_vals, (1/2) + (3/4) * alpha_vals - (21/8) * alpha_vals**2 + (333/6) * alpha_vals**3, color="magenta", label="Third Order") #Third order
plt.legend()

As you can see, the accuracy does not necessarily improve as you go to higher orders. However, the accuracy *always* improves asymptotically for smaller perturbation.